# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-1.15,100,75,0.00,CA,1682483413
1,1,badarpur,24.9000,92.6000,35.58,27,16,1.85,IN,1682483643
2,2,urengoy,65.9667,78.3667,-3.28,80,6,3.31,RU,1682483644
3,3,saint-leu,-21.1500,55.2833,27.75,85,8,2.91,RE,1682483645
4,4,fuerte olimpo,-21.0415,-57.8738,20.90,98,47,1.36,PY,1682483645


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [40]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map plot
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[((city_data_df["Cloudiness"] == 0) & (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] > 4.5))]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,i-n-salah,27.1935,2.4607,24.03,19,0,5.53,DZ,1682483667
118,118,carnarvon,-24.8667,113.6333,21.96,48,0,6.67,AU,1682483724
401,401,murzuq,25.9155,13.9184,21.17,26,0,5.34,LY,1682483925
487,487,hurghada,27.2574,33.8129,22.02,21,0,6.17,EG,1682483986


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df.drop(['City_ID','Max Temp','Cloudiness','Wind Speed','Date'],axis=1)
hotel_df = hotel_df.loc[:,['City','Country','Lat', 'Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,i-n-salah,DZ,27.1935,2.4607,19,
118,carnarvon,AU,-24.8667,113.6333,48,
401,murzuq,LY,25.9155,13.9184,26,
487,hurghada,EG,27.2574,33.8129,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories" : "accommodation",
    "limit" : 20,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
i-n-salah - nearest hotel: باجودة
carnarvon - nearest hotel: Carnarvon Motel
murzuq - nearest hotel: فندق باريس - مرزق
hurghada - nearest hotel: Sea Horse Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
34,i-n-salah,DZ,27.1935,2.4607,19,باجودة
118,carnarvon,AU,-24.8667,113.6333,48,Carnarvon Motel
401,murzuq,LY,25.9155,13.9184,26,فندق باريس - مرزق
487,hurghada,EG,27.2574,33.8129,21,Sea Horse Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)